In [26]:
import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
import PIL.Image
import google.ai.generativelanguage as glm

In [27]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [28]:
GOOGLE_API_KEY = "-------------------------------"
genai.configure(api_key=GOOGLE_API_KEY)

In [29]:
v_model = genai.GenerativeModel('gemini-1.5-flash-latest')

In [30]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-exp-1114


In [31]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from PIL import Image

In [32]:
from langchain_community.llms import Ollama
t_model = Ollama(model="llama3")

In [33]:
class MultimodalPipeline:
    def __init__(self, visual_model, text_model):
        self.visual_model = visual_model
        self.text_model = text_model

    def process_image(self, image):
        # Get description or features from the visual model
        image_description = self.visual_model.generate_content([image, "describe"])
        return image_description.text

    def process_text(self, text):
        # Directly use the text model to generate text responses
        return self.text_model(text)
        
    def generate_image_from_text(self,prompt):
        try:
            image_bytes = self.visual_model.generate(prompt=prompt)
            image = Image.open(io.BytesIO(image_bytes))
            return image
        except Exception as e:
            print(f"Error generating image: {e}")
            return Image.new("RGB", (512, 512), color="white") 
        
    def generate_detailed_prompt(self, image_description, text_description, user_input):
        detailed_prompt = (
            f"Image Analysis:\n{image_description}\n\n"    
            f"Context from Past:\n{text_input}\n\n"
            f"User Question :\n{user_input}\n\n"
            f"You are a personality psychologists Based on this, provide a response."
        )
        return detailed_prompt

    def generate_multimodal_output(self, image, text_input,user_input):
        image_description = self.process_image(image)
        text_description = self.process_text(text_input)
        detailed_prompt = self.generate_detailed_prompt(image_description, text_description, user_input)
        generated_text = self.process_text(detailed_prompt)
        generated_image = self.generate_image_from_text(generated_text)
        return generated_text, generated_image

In [34]:
image_path = 'C:\\Users\\LENOVO\\Downloads\\flower.jpg'
image = Image.open(image_path)

text_input = """
            Red color means Extrovert thinkers ,
            Yellow color means Extrovert feelers , 
            Blue color means Introverted thinkers , 
            Green color means Introverted feelers
            """

user_input =  """
              Describe my personality traits. 
              For example, If I more introverted or extroverted? How will I perform in organization and planning?
              """ 

pipeline = MultimodalPipeline(visual_model=v_model, text_model=t_model)

generated_text, generated_image = pipeline.generate_multimodal_output(image, text_input,user_input)

print("Generated Text:", generated_text)

generated_image.show() 

Error generating image: 'GenerativeModel' object has no attribute 'generate'
Generated Text: What an intriguing exercise!

Based on the vibrant blue morning glories in the image, I'm going to take a leap of faith and infer some insights about your personality traits.

As a Blue color, you likely possess qualities characteristic of Introverted Thinkers (INTJ or INFJ types). Here's what that might mean:

1. **Introversion**: You tend to be more reserved and may prefer quieter environments where you can recharge and focus on your thoughts.
2. **Thinking**: Your primary mode of thinking is logical and analytical, making decisions based on reason and evidence rather than emotions.
3. **Planning and Organization**: As an INTJ or INFJ, you're likely to be a detail-oriented planner, with a strong desire for control and structure in your life. You might excel at planning and organizing projects, tasks, and schedules.
4. **Creativity and Innovation**: While not typically associated with extraver